# Speech Act Classification using Support Vector Machines
Using a support vector machine to classify data based on the speech act.

This takes the file and extracts the sentences and labels

In [179]:
import pandas as pd

sentences = []
labels = []

data_file = "data/labeled_multiclass.xlsx"
raw_data = pd.read_excel(data_file)

raw_data.dropna(subset=['Sentence'], inplace=True)

sentences = raw_data["Sentence"]
labels = raw_data.drop(columns=["Sentence"])

## Looking at the Data Distribution
Here we will look at the output file that has the labelled data and sentences and analyse the distribution

In [180]:
raw_data.head()

,Sentence,Intel (from newspapers),Directions,Woods,Buildings,Hills and Forests,Named Locations,Fire words,Rescue words,Action words,Reasoning words
0,this is Charlie,0,0,0,0,0,0,0,0,0,0
1,Roger over,0,0,0,0,0,0,0,0,0,0
2,Bravo I didn't find anything relevant just abo...,1,0,1,1,0,1,0,0,1,0
3,nothing really relevant just saying its open I...,0,0,0,0,0,0,0,0,0,0
4,Charlie I've got advertisement feature for the...,1,0,0,0,0,0,0,0,0,0


Determine the data size and dimensions

In [181]:
raw_data.shape

(2692, 11)

In [182]:
raw_data.nunique()

Sentence                   2359
Intel (from newspapers)       2
Directions                    2
Woods                         2
Buildings                     2
Hills and Forests             2
Named Locations               2
Fire words                    2
Rescue words                  2
Action words                  2
Reasoning words               2
dtype: int64

In [183]:
print(raw_data.duplicated().sum())

333


In [184]:
raw_data.isna().sum()

Sentence                   0
Intel (from newspapers)    0
Directions                 0
Woods                      0
Buildings                  0
Hills and Forests          0
Named Locations            0
Fire words                 0
Rescue words               0
Action words               0
Reasoning words            0
dtype: int64

In [185]:
import matplotlib.pyplot as plt

label_counts = raw_data.drop(columns=["Sentence"]).sum()

# Plot distribution of label frequencies
plt.figure(figsize=(10, 6))
label_counts.plot(kind="bar")
plt.title("Distribution of Label Frequencies")
plt.xlabel("Labels")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Preprocessing of the Data
This involves removing stop words, making sure everything is lower case, removing punctuation
Vectorising based on the Tf-idf values in the data set?

## Training the Model

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2)

# TODO: try running without tfidf
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

svm_model = SVC(probability=True)  # Adjust class weights
multi_target_svm = MultiOutputClassifier(svm_model, n_jobs=-1)  # n_jobs=-1 for parallel processing
multi_target_svm.fit(X_train_tfidf, y_train)

MultiOutputClassifier(estimator=SVC(probability=True), n_jobs=-1)

### These are the values for the report that does multilabel classification with these labels:
'Intel (from newspapers)', 'Directions', 'Woods', 'Buildings', 'Hills and Forests', 'Named Locations', 'Fire words', 'Rescue words', 'Action words', 'Reasoning words'

In [187]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Perform classification with SVM, kernel=linear
y_pred = multi_target_svm.predict(X_test_tfidf)
print("Classification Report:")
report = classification_report(y_test, y_pred, zero_division=0)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(report)

Classification Report:
Accuracy: 0.70
              precision    recall  f1-score   support

           0       1.00      0.19      0.31        27
           1       1.00      0.16      0.28        31
           2       1.00      0.42      0.60        73
           3       1.00      0.65      0.79        72
           4       1.00      0.39      0.56        36
           5       0.99      0.78      0.88       153
           6       1.00      0.65      0.79        34
           7       1.00      0.76      0.86        74
           8       1.00      0.27      0.42        52
           9       1.00      0.54      0.70        54

   micro avg       1.00      0.57      0.72       606
   macro avg       1.00      0.48      0.62       606
weighted avg       1.00      0.57      0.70       606
 samples avg       0.45      0.36      0.39       606


TODO: Have something that shows how often sentences overlap with different speech acts
- Break down situation awareness, DONE
- Try to stop over-fitting (limited data set), Attempting K-Fold below
- multilabel seems to be working best
- Bert -> delve into it's language model further to tailor it to this dataset
- Why SVM?
- Report + Presentation structure

## K-Fold Training